## Clipped Relu

Notebook to make different subplots for the figure illustrating the basic clipped relu

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mayavi import mlab
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import CubicSpline
from lrr_annot.loader import Loader
from lrr_annot.analyzer import compute_winding, compute_regression, median_slope
from lrr_annot.plotter import plot_regression

## Load in sequence, compute winding number, and compute breakpoints

In [ ]:
#protid = "AT1G72840.2"
protid = "AT3G44400.1"
loader = Loader()
loader.load_single("../pdb", f"{protid}.pdb")
X = loader.structures[protid]
winding_res = compute_winding(X)
winding = winding_res["winding"]

regression_res = compute_regression(winding, n_breakpoints=2)
breakpoints = regression_res["breakpoints"]
slope = regression_res["slope"]

## Plots showing piecewise linear regression and interpro annotations

In [ ]:
plt.figure(figsize=(6, 4))
plot_regression(plt.gca(), winding, breakpoints, slope)


N = winding.size

# Define the domains
domains = [(46, 210), (231, 445), (611, 859)]
colors = ['orange', 'red', 'C2']
texts = ["TIR", "NB-ARC", "LRR"]
# Draw the domains
rectheight = 1
for domain, color, text in zip(domains, colors, texts):
    start, end = domain
    rect = patches.Rectangle((start, -rectheight*2), end - start, rectheight, facecolor=color)
    plt.gca().add_patch(rect)
    plt.text(start+(end-start)/len(text), -rectheight*1.8, text)
plt.plot([0, N], [0]*2, color='k', linewidth=1)

xticks = list(range(0, 1001, 200)) + breakpoints.tolist()
plt.xticks(xticks);
plt.yticks([0, 5, 10, 15, 20]);
plt.savefig("ClippedRelu_Plot.svg", bbox_inches='tight')

## Make Plot of Annotated Regions in 3D

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
print(colors)

In [ ]:
from mayavi import mlab

loader = Loader()
loader.load_single("../pdb", f"{protid}.pdb")

fac = 10 # Interpolation factor
X = loader.structures[protid]
spline = CubicSpline(np.linspace(0, 1, X.shape[0]), X, axis=0)
t = np.linspace(0, 1, fac*X.shape[0])
X = spline(t)

mol_plot = mlab.plot3d(X[:, 0], X[:, 1], X[:, 2], np.arange(X.shape[0]), colormap='magma', tube_radius=0.5, representation='surface')

scene = mlab.gcf().scene
scene.parallel_projection = True
scene.background = (1, 1, 1)
#mlab.savefig("out.png", size=(1000, 1000))
mlab.show()


